**Lesson 12: PCA**

The first 2/3s of the lesson is conceptual and really good. 

The big take away is that the principle component is the direction of maximal variance in the data. Like a lot of things in PCA, it sounds really complicated in English but communicated in the form of a picture it is really obvious. 

The principle components are also the directions that minimize the information loss when the data is reduced to them. 

Maximizing the variance minimizes the distance from the old points to the new transformed points. Thus, we are minimizing the information loss. 

So, PCA is an algorithm for feature selection. The great thing is that PCA can be done automatically, where the algorithm can take all features, no matter how many features there are, and come up with the principle components in the data without any human intervention. But, you don't know what the compentnets mean. 

PCA is a systematized way to transform input features into principle compenents. You then projecdt the actual data down into those principle compenents. 

All the PCs are orthogonal to each other. 

Now we start looking at real data using the data stored in "ud120-projects/final_project/"

The main sklearn function we need is the [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) module. 

Now I she is going to explain the code *finance_pca.py* that is included in the 
The basic approach is to import the module, create the analyzer, fit it to the data, and return the re-fitted data in a pca object. We see this in the body of the do_PCA() function that is defined in the code: 
```
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(data)  # only fit, not transformed
return pca
```
Then we get information out of the principle components analysis. 

There are several methods that access the data in a principle components analysis. 
```
pca.explained_variance_ratio_
first_pc = pca.components_[0]
second_pc = pca.components_[1]
```

The explained variance ratio is where the eigen values live. 

The components is a python list. This is the directional information giving you the direction of the data, the direction it is transformed by to get the new axes. 

Then we want to make a graph showing the new data. We have to create a new data set of the transformed data that we can use to plot. Remember, we have only fit the data, we have not transformed it. 
```
transformed_data = pca.transform(data)
```
Once we have the transformed data we can plot it against the original data and see how each principle component captures some of the variation in the data. 
```
for ii, jj in zip(tranformed_data, data):
    plt.scatter( first_pc[0]*ii[0], first_pc[1]*ii[0], color="r"
    plt.scatter( second_pc[0]*ii[1], second_pc[1]*ii[1], color="c"
    plt.scatter( jj[0], jj[1], color="b"
```
So, in the for loop we create a new data set that combines the principle components and the original data. Then we somehow plot the components and the data together. 

The r is going to be the first principle components times the data, the cyan will be the second principle component times the data and the blue is going to be the original data. 

Finally, we will add some labels. 
```
plt.xlabel("bonus")
plt.ylabel("long-term incentive")
plt.show()
```
Now, in presenting the analysis the first thing she does is print out the eigen values. 
```
python finance_pca.py
```
This produces a 0.9 and a 0.09 number for the two components. As usual, the first principle component summarizes about ten times as much as the second principle component does. The print of out the numbers comes from the *pca.explained_variance_ratio_* statement in the code. 

Then you get a really cool scatter plot. You get the compact red and cyan points in a compact and almost orthoganal arrangement and the blue dots representing the original data in a much more spread out pattern. The two principle components are not perfectly orthogonal because they are of vastly different scales. 

*When do you use PCA?*

Find latent features: find the bigshots at enron
Dimensionality reduction
    --visualize high dimensional data
    --reduce noise
    --make other algorithms run better

*PCA for Facial Recognition*

Eigen Faces: applying PCA to pictures of people. If you take all the pixels in people's faces and reduce them to the PCs, you can reconstruct the visual images using only the priciple components, thus, producing "Eigen Faces". 

You apply the PCA analysis to pictures of people and then let a SVM to identify people. 

There are tons of input features. Each pixel is an input. But there are only a few underlying big differences, where are the eyes, how long is the nose? 

So, the mini-project is to go through the Eigen Faces code and get it to work. I think I may postpone the mini-project on this lesson to get going on the Enron project sooner. 


In [ ]:
# %load eigenfaces.py
"""
===================================================
Faces recognition example using eigenfaces and SVMs
===================================================

The dataset used in this example is a preprocessed excerpt of the
"Labeled Faces in the Wild", aka LFW_:

  http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz (233MB)

  .. _LFW: http://vis-www.cs.umass.edu/lfw/

  original source: http://scikit-learn.org/stable/auto_examples/applications/face_recognition.html

"""



print __doc__

from time import time
import logging
import pylab as pl
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.datasets import fetch_lfw_people
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import RandomizedPCA
from sklearn.svm import SVC

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')


###############################################################################
# Download the data, if not already on disk and load it as numpy arrays
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape
np.random.seed(42)

# for machine learning we use the data directly (as relative pixel
# position info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print "Total dataset size:"
print "n_samples: %d" % n_samples
print "n_features: %d" % n_features
print "n_classes: %d" % n_classes


###############################################################################
# Split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

###############################################################################
# Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled
# dataset): unsupervised feature extraction / dimensionality reduction
n_components = 150

print "Extracting the top %d eigenfaces from %d faces" % (n_components, X_train.shape[0])
t0 = time()
pca = RandomizedPCA(n_components=n_components, whiten=True).fit(X_train)
print "done in %0.3fs" % (time() - t0)

eigenfaces = pca.components_.reshape((n_components, h, w))

print "Projecting the input data on the eigenfaces orthonormal basis"
t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print "done in %0.3fs" % (time() - t0)


###############################################################################
# Train a SVM classification model

print "Fitting the classifier to the training set"
t0 = time()
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }
# for sklearn version 0.16 or prior, the class_weight parameter value is 'auto'
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(X_train_pca, y_train)
print "done in %0.3fs" % (time() - t0)
print "Best estimator found by grid search:"
print clf.best_estimator_


###############################################################################
# Quantitative evaluation of the model quality on the test set

print "Predicting the people names on the testing set"
t0 = time()
y_pred = clf.predict(X_test_pca)
print "done in %0.3fs" % (time() - t0)

print classification_report(y_test, y_pred, target_names=target_names)
print confusion_matrix(y_test, y_pred, labels=range(n_classes))


###############################################################################
# Qualitative evaluation of the predictions using matplotlib

def plot_gallery(images, titles, h, w, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    pl.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    pl.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        pl.subplot(n_row, n_col, i + 1)
        pl.imshow(images[i].reshape((h, w)), cmap=pl.cm.gray)
        pl.title(titles[i], size=12)
        pl.xticks(())
        pl.yticks(())


# plot the result of the prediction on a portion of the test set

def title(y_pred, y_test, target_names, i):
    pred_name = target_names[y_pred[i]].rsplit(' ', 1)[-1]
    true_name = target_names[y_test[i]].rsplit(' ', 1)[-1]
    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)

prediction_titles = [title(y_pred, y_test, target_names, i)
                         for i in range(y_pred.shape[0])]

plot_gallery(X_test, prediction_titles, h, w)

# plot the gallery of the most significative eigenfaces

eigenface_titles = ["eigenface %d" % i for i in range(eigenfaces.shape[0])]
plot_gallery(eigenfaces, eigenface_titles, h, w)

pl.show()
